<a href="https://colab.research.google.com/github/BLOSSOM1994/CarDivar-/blob/main/train_mask_car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Mask RCNN into Google Colab

In [1]:
!git clone https://github.com/BLOSSOM1994/Mask_RCNN.git

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 973, done.
remote: Total 973 (delta 0), reused 0 (delta 0), pack-reused 973
Receiving objects: 100% (973/973), 125.23 MiB | 31.35 MiB/s, done.
Resolving deltas: 100% (572/572), done.


## Set up the GPU Environment
In Colabs go to:  
Edit > Notebook Settings  
Set the hardware accelerator to 'GPU' and reload the notebook. 

In [2]:
#RUN GPU
%tensorflow_version 1.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 1.x selected.
Found GPU at: /device:GPU:0


## Move into the Mask RCNN directory and install the package

In [3]:
import os
os.chdir('Mask_RCNN')

In [4]:
!ls

assets	LICENSE      mrcnn	requirements.txt  setup.cfg
images	MANIFEST.in  README.md	samples		  setup.py


In [5]:
!python3 setup.py install;

/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:645: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:645: UserWarning: Usage of dash-separated 'license-file' will not be supported in future versions. Please use the underscore name 'license_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:645: UserWarning: Usage of dash-separated 'requirements-file' will not be supported in future versions. Please use the underscore name 'requirements_file' instead
  % (opt, underscore_opt))
running install
running bdist_egg
running egg_info
creating mask_rcnn.egg-info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
writing manifest file 'ma

## Load Modules

In [6]:
import mrcnn
import mrcnn.model as modellib
from mrcnn.config import Config
from mrcnn import utils
from mrcnn.model import MaskRCNN

import numpy as np
import colorsys
import argparse
import random
import os
import sys
import time
import json
import skimage
import datetime
from keras.models import load_model


Using TensorFlow backend.


## Link Google Drive and set up directories

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Define the Config and Dataset classes

In [17]:
class myMaskRCNNConfig(Config):
    NAME = "MaskRCNN_config"
 
    # set the number of GPUs to use along with the number of images
    # per GPU
    # Colabs provides a 12GB GPU which should fit 2 images
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2
 
    # number of classes (we would normally add +1 for the background)
    # COVID + BG
    NUM_CLASSES = 1+1
   
    # Number of training steps per epoch (set to number of training samples)
    STEPS_PER_EPOCH = 80
    
    # Learning rate (reduce to stop NaN losses)
    LEARNING_RATE=0.001
    
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    
    # setting Max ground truth instances
    MAX_GT_INSTANCES=10
    
config = myMaskRCNNConfig()
config.display()

check=[]
class NissanDataset(utils.Dataset):

    def load_dataset(self, dataset_dir):
        """
        Load the images and annotations from Google Drive
        The mask coordinates are save in 'polygons'
        """

        self.add_class("Nissan", 1, "Nissan")

        # Load annotations
        # VGG Image Annotator saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        annotations = json.load(open(os.path.join(homedir, "nissan_json.json")))
        annotations = list(annotations.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        for a in annotations:
            if a['filename'] in os.listdir(dataset_dir):
                # Get the x, y coordinates of points of the polygons that make up
                # the outline of each object instance. These are stored in the
                # shape_attributes (see json format above)
        
                if type(a['regions']) is dict:
                        polygons = [r['shape_attributes'] for r in a['regions'].values()]
                else:
                        polygons = [r['shape_attributes'] for r in a['regions']]
        
                # load_mask() needs the image size to convert polygons to masks.
                # Unfortunately, VIA doesn't include it in JSON, so we must read
                # the image. This is only managable since the dataset is tiny.
                image_path = os.path.join(dataset_dir, a['filename'])
                image = skimage.io.imread(image_path)
                height, width = image.shape[:2]
    
                self.add_image(
                    "Nissan",
                    image_id=a['filename'],
                    path=image_path,
                    width=width, height=height,
                    polygons=polygons)

    def load_mask(self, image_id):
        """
        Generate instance masks for an image.
        
        Returns:
        
        masks: A bool array of shape [height, width, instance count] with
               one mask per instance.
        
        class_ids: a 1D array of class IDs of the instance masks.
        
        """
        # If not a Nissan dataset image, delegate to background class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "Nissan":
            print('not a Nissan!')
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)

        class_id=[]
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1
            class_id.append(self.class_names.index('Nissan'))

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask, np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "Nissan":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [14]:
homedir='/content/drive/MyDrive/Colab Notebooks/Data/'
datadir=homedir+'Car_test/'

In [15]:
os.listdir(datadir)

['train', 'test', 'predict']

In [11]:
#files need to be accessed using os !ls does not work.
#os.listdir(homedir)

# Prepare the Training and Test Sets

In [18]:
# prepare train set
train_set = NissanDataset()
train_set.load_dataset(datadir+'train')
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# prepare test/val set
test_set = NissanDataset()
test_set.load_dataset(datadir+'test')
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

Train: 20
Test: 3


# Prepare the Training Model:

In [20]:
print("Loading Mask R-CNN model...")
model = modellib.MaskRCNN(mode="training", config=config, model_dir='./')
homed='/content/drive/MyDrive/Colab Notebooks'
#load the weights for COCO
model.load_weights(homed+'/mask_rcnn_coco.h5', 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

Loading Mask R-CNN model...


## Train the Model:
If you begin to encounter NaNs in your loss values, try lowering the learning rate by 4 or so times. 

In [21]:
# train heads with higher lr to speedup the learning

model.train(train_set, test_set, learning_rate=2*config.LEARNING_RATE, epochs=25, layers='heads')
history = model.keras_model.history.history


Starting at epoch 0. LR=0.002

Checkpoint Path: ./maskrcnn_config20210403T1603/mask_rcnn_maskrcnn_config_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4   

/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/tensorflow-1.15.2/python3.7/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/25
80/80 [==============================] - 74s 926ms/step - loss: 0.9332 - rpn_class_loss: 0.0078 - rpn_bbox_loss: 0.2200 - mrcnn_class_loss: 0.0353 - mrcnn_bbox_loss: 0.3743 - mrcnn_mask_loss: 0.2958 - val_loss: 0.3321 - val_rpn_class_loss: 0.0056 - val_rpn_bbox_loss: 0.0998 - val_mrcnn_class_loss: 0.0161 - val_mrcnn_bbox_loss: 0.0968 - val_mrcnn_mask_loss: 0.0834

Epoch 2/25
80/80 [==============================] - 57s 709ms/step - loss: 0.4133 - rpn_class_loss: 0.0058 - rpn_bbox_loss: 0.1934 - mrcnn_class_loss: 0.0157 - mrcnn_bbox_loss: 0.1105 - mrcnn_mask_loss: 0.0879 - val_loss: 0.2906 - val_rpn_class_loss: 0.0060 - val_rpn_bbox_loss: 0.1003 - val_mrcnn_class_loss: 0.0136 - val_mrcnn_bbox_loss: 0.0909 - val_mrcnn_mask_loss: 0.0700
Epoch 3/25
80/80 [==============================] - 57s 713ms/step - loss: 0.3319 - rpn_class_loss: 0.0051 - rpn_bbox_loss: 0.1853 - mrcnn_class_loss: 0.0138 - mrcnn_bbox_loss: 0.0583 - mrcnn_mask_loss: 0.0695 - val_loss: 0.2300 - val_rpn_class_

## Check the Output
Find the output directory name.  
In this example it is: 'maskrcnn_config20200218T1359'

In [22]:
!ls

assets	images	     maskrcnn_config20210403T1603  README.md	     setup.cfg
build	LICENSE      mask_rcnn.egg-info		   requirements.txt  setup.py
dist	MANIFEST.in  mrcnn			   samples


In [24]:
#Move to the latest model directory
os.chdir('./maskrcnn_config20210403T1603')

## Save Model Weights and History in Google Drive:
Find the name of the latest model weights to save

In [25]:
!ls -lhrt

total 6.1G
-rw-r--r-- 1 root root 245M Apr  3 16:07 mask_rcnn_maskrcnn_config_0001.h5
-rw-r--r-- 1 root root 245M Apr  3 16:08 mask_rcnn_maskrcnn_config_0002.h5
-rw-r--r-- 1 root root 245M Apr  3 16:09 mask_rcnn_maskrcnn_config_0003.h5
-rw-r--r-- 1 root root 245M Apr  3 16:10 mask_rcnn_maskrcnn_config_0004.h5
-rw-r--r-- 1 root root 245M Apr  3 16:12 mask_rcnn_maskrcnn_config_0005.h5
-rw-r--r-- 1 root root 245M Apr  3 16:13 mask_rcnn_maskrcnn_config_0006.h5
-rw-r--r-- 1 root root 245M Apr  3 16:14 mask_rcnn_maskrcnn_config_0007.h5
-rw-r--r-- 1 root root 245M Apr  3 16:15 mask_rcnn_maskrcnn_config_0008.h5
-rw-r--r-- 1 root root 245M Apr  3 16:16 mask_rcnn_maskrcnn_config_0009.h5
-rw-r--r-- 1 root root 245M Apr  3 16:17 mask_rcnn_maskrcnn_config_0010.h5
-rw-r--r-- 1 root root 245M Apr  3 16:18 mask_rcnn_maskrcnn_config_0011.h5
-rw-r--r-- 1 root root 245M Apr  3 16:19 mask_rcnn_maskrcnn_config_0012.h5
-rw-r--r-- 1 root root 245M Apr  3 16:20 mask_rcnn_maskrcnn_config_0013.h5
-rw-r--r-- 1 r

In [26]:
now_str='20210403'

try:
  os.makedirs(homedir+'saved')
except:
  print('Save directory exists!')

weights_fname =homedir+'saved/weights_Nissan_'+now_str+'.h5'
history_fname =homedir+'saved/history_Nissan_'+now_str+'.json'

model.keras_model.save_weights(weights_fname)

with open(history_fname,"w") as f:
   f.write(json.dumps(str(history)))
   